In [17]:
import torch, clip, os
from PIL import Image
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
images_path, feats = "data/cropped/", []

for i in tqdm(os.listdir(images_path)):
    img_path = os.path.join(images_path, i)
    image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        feats.append(model.encode_image(image))

feats = torch.vstack(feats)
# feats.shape = (862, 512)

100%|██████████| 862/862 [00:25<00:00, 34.27it/s]


In [1]:
import numpy as np

def get_results(logger):
    auc, sns, acc, spc = [], [], [], []
    with open(logger, "r") as f:
        for line in f.readlines():
            if line.startswith("AUC"):
                n = line.split(":")[1][1:5]
                auc.append(float(n))
            if line.startswith("Acc"):
                n = line.split(":")[1][1:5]
                acc.append(float(n))
            if line.startswith("Sen"):
                n = line.split(":")[1][1:5]
                sns.append(float(n))
            if line.startswith("Spe"):
                n = line.split(":")[1][1:5]
                spc.append(float(n))
    return auc, acc, sns, spc

def calc_ci(distr, p):
    dmean = np.mean(distr)
    ci_do = np.percentile(distr, p/2)
    ci_up = np.percentile(distr, 100-p/2)
    return "{:.1f}% ({}% CI, {:.1f} to {:.1f})".format(dmean, 100-p, ci_do, ci_up)

In [5]:
auc, acc, sns, spc = get_results("results/logger_41.txt")
print("AUC =", calc_ci(auc, p=5))
print("Acc =", calc_ci(acc, p=5))
print("Sns =", calc_ci(sns, p=5))
print("Spc =", calc_ci(spc, p=5))

AUC = 67.3% (95% CI, 58.2 to 75.8)
Acc = 67.6% (95% CI, 61.4 to 78.3)
Sns = 68.6% (95% CI, 45.4 to 79.5)
Spc = 66.1% (95% CI, 52.2 to 83.4)


In [3]:
auc, acc, sns, spc = get_results("results/logger_40.txt")
print("AUC =", calc_ci(auc, p=5))
print("Acc =", calc_ci(acc, p=5))
print("Sns =", calc_ci(sns, p=5))
print("Spc =", calc_ci(spc, p=5))

AUC = 81.1% (95% CI, 74.9 to 88.3)
Acc = 84.3% (95% CI, 75.4 to 89.4)
Sns = 74.9% (95% CI, 66.5 to 86.3)
Spc = 87.3% (95% CI, 77.4 to 92.2)
